In [2]:
import pandas as pd
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset original
df = pd.read_csv("../data/flight_delays_sample.csv") # usa aquí el nombre de tu archivo original

# ============================
# 🧹 LIMPIEZA Y FEATURES
# ============================

# Convertir fechas
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], errors='coerce')

# Crear columnas de fecha
df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek
df['MONTH'] = df['FL_DATE'].dt.month
df['DAY'] = df['FL_DATE'].dt.day

# Columnas horarias
df['DEP_HOUR'] = df['CRS_DEP_TIME'] // 100
df['ARR_HOUR'] = df['CRS_ARR_TIME'] // 100

# Variable objetivo: retrasos mayores a 30 minutos
df['IS_DELAYED'] = (df['ARR_DELAY'] > 30).astype(int)

# Variables temporales adicionales
df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([5, 6]).astype(int)
df['IS_SUMMER'] = df['MONTH'].isin([6, 7, 8]).astype(int)

# Eliminar la columna de fecha para evitar errores con modelos
df = df.drop(columns=['FL_DATE'])

# ============================
# 🧠 ENRIQUECIMIENTO DE DATOS
# ============================

# Retrasos medios por aerolínea, origen y destino
airline_delay = df.groupby('AIRLINE')['ARR_DELAY'].mean()
origin_delay = df.groupby('ORIGIN')['ARR_DELAY'].mean()
dest_delay = df.groupby('DEST')['ARR_DELAY'].mean()

df['AIRLINE_MEAN_DELAY'] = df['AIRLINE'].map(airline_delay)
df['ORIGIN_MEAN_DELAY'] = df['ORIGIN'].map(origin_delay)
df['DEST_MEAN_DELAY'] = df['DEST'].map(dest_delay)

print("✅ Dataset enriquecido con variables de contexto")


✅ Dataset enriquecido con variables de contexto


In [3]:
# Cargar base de datos pública de aeropuertos
airports = pd.read_csv(
    'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',
    header=None,
    names=['id','name','city','country','IATA','ICAO','lat','lon','alt','tz','dst','tz_db','type','source']
)

# Filtrar solo aeropuertos de Estados Unidos
airports_us = airports[(airports['country'] == 'United States') & (airports['IATA'].notnull())]

# Unir coordenadas de origen
df = df.merge(
    airports_us[['IATA', 'lat', 'lon']],
    how='left',
    left_on='ORIGIN',
    right_on='IATA'
)
df.rename(columns={'lat': 'lat_origin', 'lon': 'lon_origin'}, inplace=True)
df.drop(columns=['IATA'], inplace=True)

# Unir coordenadas de destino
df = df.merge(
    airports_us[['IATA', 'lat', 'lon']],
    how='left',
    left_on='DEST',
    right_on='IATA'
)
df.rename(columns={'lat': 'lat_dest', 'lon': 'lon_dest'}, inplace=True)
df.drop(columns=['IATA'], inplace=True)

print("✅ Coordenadas de aeropuertos unidas correctamente.")


✅ Coordenadas de aeropuertos unidas correctamente.


In [6]:
# Crear una muestra (para que no pese demasiado)
df_final = df.sample(300_000, random_state=42)

# Guardar CSV final listo para Tableau
df_final.to_csv('flight_delays_tableau_clean.csv', index=False, encoding='utf-8', sep=',', lineterminator='\n')


print("📁 Archivo guardado como 'flight_delays_tableau.csv'")


📁 Archivo guardado como 'flight_delays_tableau.csv'


In [6]:
import pandas as pd
df = pd.read_csv("../data/flight_delays_tableau_clean.csv", sep=';', encoding='utf-8-sig')

In [11]:
import csv
df.to_csv(
    'flight_delays_tableau_ready.csv',
    index=False,
    sep=';',                 
    encoding='utf-8-sig',
    lineterminator='\n',
    quoting=csv.QUOTE_ALL
)

print("✅ Archivo final guardado como 'flight_delays_tableau_ready.csv' 🎉")

✅ Archivo final guardado como 'flight_delays_tableau_ready.csv' 🎉
